In [ ]:
vol_garch_2_2 = res_garch_2_2.conditional_volatility
vol_egarch    = res_egarch.conditional_volatility
vol_gjr       = res_gjr.conditional_volatility


In [ ]:
data_filtered["vol_garch22"] = vol_garch_2_2
data_filtered["vol_egarch"]  = vol_egarch
data_filtered["vol_gjr"]     = vol_gjr



In [ ]:
import pandas as pd

if isinstance(data_filtered.columns, pd.MultiIndex):
    data_filtered.columns = data_filtered.columns.get_level_values(0)

print(data_filtered.columns)



In [ ]:
cols = ["log_return", "RV_rolling10", "vol_garch22", "vol_egarch", "vol_gjr"]

data_filtered = data_filtered.dropna(subset=cols).copy()

for c in cols:
    print(c, data_filtered[c].shape)


In [ ]:
n = len(data_filtered)
train_size = int(0.9 * n)
train_data = data_filtered.iloc[:train_size]
val_data   = data_filtered.iloc[train_size:]

In [ ]:
# Features (entrées)
features_cols = ["log_return", "RV_rolling10", "vol_garch22", "vol_egarch", "vol_gjr"]

X_train_raw = train_data[features_cols].values
y_train_raw = train_data["RV_rolling10"].values.reshape(-1, 1)

X_val_raw   = val_data[features_cols].values
y_val_raw   = val_data["RV_rolling10"].values.reshape(-1, 1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train_raw)
y_train_scaled = scaler_y.fit_transform(y_train_raw)

X_val_scaled   = scaler_X.transform(X_val_raw)
y_val_scaled   = scaler_y.transform(y_val_raw)


In [ ]:
import numpy as np

def create_sequences_multi(X, y, lookback=60):
    X_seq, y_seq = [], []
    for i in range(lookback, len(X)):
        X_seq.append(X[i-lookback:i, :])  # 60 jours de toutes les features
        y_seq.append(y[i, 0])             # la cible au jour i
    return np.array(X_seq), np.array(y_seq)

lookback = 60

X_train, y_train = create_sequences_multi(X_train_scaled, y_train_scaled, lookback)
X_val,   y_val   = create_sequences_multi(X_val_scaled,   y_val_scaled,   lookback)

print("X_train shape :", X_train.shape)  # (n_samples, 60, n_features)
print("y_train shape :", y_train.shape)
print("X_val shape   :", X_val.shape)
print("y_val shape   :", y_val.shape)

In [ ]:
print("X_train shape :", X_train.shape)
print("X_val shape   :", X_val.shape)



In [ ]:
n_timesteps = X_train.shape[1]   # normalement 60
n_features  = X_train.shape[2]   # normalement 5


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(n_timesteps, n_features)),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="mse",
    metrics=["mae"]
)

history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    shuffle=False,
    verbose=1
)


In [ ]:
def create_sequences_multi(X, y, lookback=60):
    X_seq, y_seq = [], []
    for i in range(lookback, len(X)):
        X_seq.append(X[i-lookback:i, :])  # bien ":" ici, on garde toutes les colonnes
        y_seq.append(y[i, 0])
    return np.array(X_seq), np.array(y_seq)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label="Loss (train)", color="blue")
plt.title("Évolution du MSE – Modèle Hybride GARCH–LSTM")
plt.xlabel("Époques")
plt.ylabel("Loss (MSE)")
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Prédictions sur le jeu de validation (en échelle normalisée)
y_pred_scaled = model.predict(X_val)

In [ ]:
# Revenir à l'échelle originale
y_pred = scaler_y.inverse_transform(y_pred_scaled)                 # prédictions
y_val_original = scaler_y.inverse_transform(y_val.reshape(-1, 1))  # vraie volatilité


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

mse = mean_squared_error(y_val_original, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_val_original, y_pred)
r2 = r2_score(y_val_original, y_pred)

print("📉 Évaluation du modèle HYBRIDE GARCH–LSTM (jeu de validation) :")
print(f"MSE  = {mse:.6f}")
print(f"RMSE = {rmse:.6f}")
print(f"MAE  = {mae:.6f}")
print(f"R²   = {r2:.4f}")

In [ ]:
plt.figure(figsize=(12,5))
plt.plot(y_val_original, label="Volatilité réalisée lissée (réelle)", color="black", linewidth=1.5)
plt.plot(y_pred,        label="Volatilité prédite (Hybride GARCH–LSTM)", color="crimson", linewidth=1.5)
plt.title("Prévision de la volatilité – Modèle Hybride GARCH–LSTM (validation)")
plt.xlabel("Temps (jours)")
plt.ylabel("Volatilité réalisée (%)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.5)
plt.tight_layout()
plt.show()